### Задание 1.
Обязательная часть
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

Дополнительная часть (необязательная)
Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import time

In [ ]:
head_url = 'https://habr.com'                                                           # голова адреса страницы
url = head_url + '/ru/all/'                                                             # адрес страницы
keywords = ['python', 'парсинг']                                                        # список слов для поиска
headers = {'User-Agent': 
           'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36 Edg/91.0.864.70'
          }
keywords_search = '|'.join(keywords).lower()                                            # строка поиска в формате RegExp
params = {}                                                                             # словарь параметров
df_result = pd.DataFrame()                                                              # DataFrame результатов поиска

def find_item(head_url, post):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36 Edg/91.0.864.70'}
    header = post.find('a', class_='tm-article-snippet__title-link')                    # делим пост на заголовок
    post_header = header.text
    post_review = post.find('div', class_='article-formatted-body').text                #                        и обзор
    post_link   = head_url + header.attrs.get('href')                                   # из заголовка получаем ссылку  
    req = requests.get(post_link, headers=headers)                                      # по ссылке запрашиваем содержимое
    if req.ok:                                                                          # если содержимое получено
        soup =  BeautifulSoup(req.text, 'html.parser')                                  
        post_body = soup.find('div', class_='tm-article-body').text                     # то достаем из него текст статьи
    else:
        post_body =''
    if (re.search(fr"{keywords_search}",                                                # если находим ключевые слова
                  post_header                                                           # в заголовке
                  + post_review                                                         # обзоре
                  + post_body)):                                                        # или самом тексте статьи,
        post_date = (post.find('span', class_='tm-article-snippet__datetime-published') # то получаем дату из поста    
                     .time                                                              
                     .attrs                                                             
                     .get('datetime'))                                                                    
        return {'Дата': post_date,                                                      # и возвращаем результат в виде словаря
                'Заголовок': str(post_header).strip(), 
                'Ссылка': post_link, 
                'Текст статьи': str(post_body).strip()}
    else:                                                                               # если совпадений не найдено,
        return False                                                                    # то возвращаем False
    
req = requests.get(url, headers=headers)                                                # читаем содержимое с заданной страницы
soup = BeautifulSoup(req.text, 'html.parser') 
page_count = int(soup.find_all('a', class_="tm-pagination__page")[-1].text.strip())     # ищем число страниц
for page in range(1, page_count + 1):                                                   # проходим по каждой странице 
    params['page'] = page                                                               # в параметры передаем номер текущей страницы
    req = requests.get(url, params, headers=headers)                                    # запрашиваем содержимое текущей страницы
    if req.ok:                                                                          # если результат запроса удачный
        soup = BeautifulSoup(req.text, 'html.parser')                                   # читаем текст в суп
        posts = soup.find_all('div', class_='tm-article-snippet')                       # разбиваем на посты
        for post in posts:                                                              # проходим по каждому посту
            item = find_item(head_url, post)                                            # передаем шапку адреса и пост, получаем словарь или False
            time.sleep(0.3)
            if item:                                                                    # если вернулся словарь, то добавляем в DF
                df_result = df_result.append(item, ignore_index=True)

In [ ]:
df_result

### Задание 2.
Обязательная часть
Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

Дополнительная часть (необязательная)
Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.
Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.get

GROUP = 'netology'  
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ
В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>

In [1]:
import requests
import pandas as pd

In [2]:
url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 YaBrowser/21.6.1.274 Yowser/2.5 Safari/537.36',

           'Vaar-Version': '0'
Accept: application/json, text/plain, */*
Accept-Encoding: gzip, deflate, br
Accept-Language: ru,en;q=0.9
Connection: keep-alive
Content-Length: 31
Content-Type: application/json;charset=UTF-8
Host: identityprotection.avast.com
Origin: https://www.avast.com
Referer: https://www.avast.com/
sec-ch-ua: " Not;A Brand";v="99", "Yandex";v="91", "Chromium";v="91"
sec-ch-ua-mobile: ?0
Sec-Fetch-Dest: empty
Sec-Fetch-Mode: cors
Sec-Fetch-Site: same-site
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 YaBrowser/21.6.1.274 Yowser/2.5 Safari/537.36
Vaar-Header-App-Build-Version: 1.0.0
Vaar-Header-App-Product: hackcheck-web-avast
Vaar-Header-App-Product-Name: hackcheck-web-avast
Vaar-Version: 0           
           
           
           
           
           
           
          }
EMAIL = ['xxx@x.ru', 'yyy@y.com']
        
jsn ={'emailAddresses': EMAIL}
res = requests.post(url, json=jsn, headers=headers)
if res.status_code == '200':
    df_breaches = pd.DataFrame(res.json()['breaches']).T.loc[:, ['publishDate', 'site', 'description']]
else: 
    print(res)

<Response [400]>


In [ ]:
df_breaches